#Task 6: Automatic cyberbullying detection

##Task 6-1: Harmful vs non-harmful

###Getting data and building models

In [121]:
%%capture
!wget http://2019.poleval.pl/task6/task_6-1.zip -O  task_6-1.zip
!unzip task_6-1
%rm task_6-1.zip

In [122]:
#Get stopwords
%%capture
!wget https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt

In [123]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.layers import Embedding
from keras.layers import SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from keras import layers
from tensorflow.keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [124]:
tags=[]
tweets=[]
with open('/content/training_set_clean_only_tags.txt','r') as f:        
     for row in f.readlines():
       tags.append(row.rstrip("\n"))
with open('/content/training_set_clean_only_text.txt','r') as f:        
     for row in f.readlines():
       tweets.append(row.rstrip("\n"))
train_df = pd.DataFrame({'tweet':tweets, 'tag':tags})

In [125]:
stop =[]
with open("/content/polish.stopwords.txt", "r") as a_file:
  for line in a_file:
    stopword = line.strip()
    stop.append(stopword)

In [126]:
train_df["tweet"] = train_df["tweet"].str.lower()
train_df['tweet_without_stopwords'] = train_df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [127]:
train_df['clean_tweet']=''

In [128]:
#Removing link
train_df['clean_tweet'] = train_df['tweet_without_stopwords'].str.replace('https?://\S+|www\.\S+', '')

In [129]:
#Removing mails and tagged people
train_df['clean_tweet'] = train_df['clean_tweet'].str.replace('\S*@\S*\s?', '')

In [130]:
#Removing punctuation
train_df['clean_tweet'] = train_df['clean_tweet'].str.replace('[^\w\s]', '')

In [131]:
#Removing numericals
train_df['clean_tweet'] = train_df['clean_tweet'].str.replace('[0-9]+', '')

In [132]:
#Remove repeating words
train_df['clean_tweet'] = train_df['clean_tweet'].str.replace('(.)1+', r'1')

In [133]:
train_df=train_df[['clean_tweet','tag']]
train_df.head()

,clean_tweet,tag
0,faworytem tytułu cracovia zobaczymy typ sprawdzi,0
1,brawo daria kibic dobre złe,0
2,super polski premier składa kwiaty grobach kol...,0
3,musi innej drogi mamy,0
4,odrzut natychmiastowy kwaśna mina problem,0


In [134]:
X = train_df['clean_tweet']
Y = train_df['tag']

In [135]:
Y = np.array(Y)
Y = to_categorical(Y, 2, dtype="float32")

In [136]:
max_words = 5000
max_len = 200
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
tweets = pad_sequences(sequences, maxlen=max_len)

In [137]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tweets,Y, random_state=0)

#### **LSTM**

In [ ]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 20))
model1.add(layers.LSTM(15,dropout=0.5))
model1.add(layers.Dense(2,activation='softmax'))
model1.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])
history = model1.fit(X_train, y_train, epochs=20,validation_data=(X_test, y_test),verbose=0)

####  **1D ConvNets**

In [139]:
model2=Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len))
model2.add(layers.Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model2.add(layers.MaxPooling1D(5))
model2.add(layers.Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model2.add(layers.GlobalMaxPooling1D())
model2.add(layers.Dense(2,activation='softmax'))
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
history2 = model2.fit(X_train, y_train, epochs=20,validation_data=(X_test, y_test),verbose=0)

### Evaluation and comparison of models

In [140]:
%%capture
!wget http://2019.poleval.pl/task6/task6_test.zip -O  task_6-1.zip
!unzip task_6-1
%rm task_6-1.zip

In [141]:
tags=[]
tweets=[]
with open('/content/Task6/task 01/test_set_clean_only_tags.txt','r') as f:        
     for row in f.readlines():
       tags.append(row.rstrip("\n"))
with open('/content/Task6/task 01/test_set_clean_only_text.txt','r') as f:        
     for row in f.readlines():
       tweets.append(row.rstrip("\n"))
test_df = pd.DataFrame({'tweet':tweets, 'tag':tags})
X = test_df['tweet']
y = test_df['tag']
max_words = 5000
max_len = 200
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
test = pad_sequences(sequences, maxlen=max_len)
Y = np.array(y)
Y = to_categorical(Y, 2, dtype="float32")

In [142]:
#LSTM evaluation
model1.evaluate(test,Y)

32/32 [==============================] - 1s 16ms/step - loss: 0.6262 - accuracy: 0.8400


[0.6262050271034241, 0.8399999737739563]

In [143]:
#1D ConvNets evaluation
model2.evaluate(test,Y)

32/32 [==============================] - 0s 6ms/step - loss: 0.4191 - acc: 0.8660


[0.4191458225250244, 0.8659999966621399]

In [144]:
%rm /content/training_set_clean_only_tags.txt
%rm /content/training_set_clean_only_text.txt

##Task 6-2: Type of harmfulness

###Getting data and building models

In [145]:
%%capture
!wget http://2019.poleval.pl/task6/task_6-2.zip -O  task_6-2.zip
!unzip task_6-2
%rm task_6-2.zip

In [146]:
tags=[]
tweets=[]
with open('/content/training_set_clean_only_tags.txt','r') as f:        
     for row in f.readlines():
       tags.append(row.rstrip("\n"))
with open('/content/training_set_clean_only_text.txt','r') as f:        
     for row in f.readlines():
       tweets.append(row.rstrip("\n"))
train_df = pd.DataFrame({'tweet':tweets, 'tag':tags})

In [147]:
stop =[]
with open("/content/polish.stopwords.txt", "r") as a_file:
  for line in a_file:
    stopword = line.strip()
    stop.append(stopword)

In [148]:
train_df["tweet"] = train_df["tweet"].str.lower()
train_df['tweet_without_stopwords'] = train_df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [149]:
train_df['clean_tweet']=''

In [150]:
#Removing link
train_df['clean_tweet'] = train_df['tweet_without_stopwords'].str.replace(r'https?://\S+|www\.\S+', '')

In [151]:
#Removing mails and tagged people
train_df['clean_tweet'] = train_df['clean_tweet'].str.replace('\S*@\S*\s?', '')

In [152]:
#Removing punctuation
train_df['clean_tweet'] = train_df['clean_tweet'].str.replace('[^\w\s]', '')

In [153]:
#Removing numericals
train_df['clean_tweet'] = train_df['clean_tweet'].str.replace('[0-9]+', '')

In [154]:
#Remove repeating words
train_df['clean_tweet'] = train_df['clean_tweet'].str.replace('(.)1+', '1')

In [155]:
train_df=train_df[['clean_tweet','tag']]
train_df.head()

,clean_tweet,tag
0,faworytem tytułu cracovia zobaczymy typ sprawdzi,0
1,brawo daria kibic dobre złe,0
2,super polski premier składa kwiaty grobach kol...,0
3,musi innej drogi mamy,0
4,odrzut natychmiastowy kwaśna mina problem,0


In [156]:
X = train_df['clean_tweet']
Y = train_df['tag']

In [157]:
Y = np.array(Y)
Y = to_categorical(Y, 3, dtype="float32")

In [158]:
max_words = 5000
max_len = 200
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
tweets = pad_sequences(sequences, maxlen=max_len)

In [159]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tweets,Y, random_state=0)

### **LSTM**

In [160]:
model3 = Sequential()
model3.add(layers.Embedding(max_words, 20))
model3.add(layers.LSTM(15,dropout=0.5))
model3.add(layers.Dense(3,activation='softmax'))
model3.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])
history3 = model3.fit(X_train, y_train, epochs=20,validation_data=(X_test, y_test),verbose=0)

### **1D ConvNets**

In [161]:
model4=Sequential()
model4.add(layers.Embedding(max_words, 40, input_length=max_len))
model4.add(layers.Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model4.add(layers.MaxPooling1D(5))
model4.add(layers.Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model4.add(layers.GlobalMaxPooling1D())
model4.add(layers.Dense(3,activation='softmax'))
model4.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
history4 = model4.fit(X_train, y_train, epochs=20,validation_data=(X_test, y_test),verbose=0)

### Evaluation and comparison of models

In [162]:
tags=[]
tweets=[]
with open('/content/Task6/task 02/test_set_only_tags.txt','r') as f:        
     for row in f.readlines():
       tags.append(row.rstrip("\n"))
with open('/content/Task6/task 02/test_set_only_text.txt','r') as f:        
     for row in f.readlines():
       tweets.append(row.rstrip("\n"))
print(len(tags),len(tweets))
test_df = pd.DataFrame({'tweet':tweets, 'tag':tags})
X2 = test_df['tweet']
y2 = test_df['tag']
max_words = 5000
max_len = 200
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X2)
sequences = tokenizer.texts_to_sequences(X2)
test2 = pad_sequences(sequences, maxlen=max_len)
Y2 = np.array(y2)
Y2 = to_categorical(Y2, 3, dtype="float32")

1000 1000


In [163]:
#LSTM evaluation
model3.evaluate(test2,Y2)

32/32 [==============================] - 0s 15ms/step - loss: 0.4342 - accuracy: 0.8400


[0.4341752827167511, 0.8399999737739563]

In [164]:
#1D ConvNets evaluation
model4.evaluate(test2,Y2)

32/32 [==============================] - 0s 6ms/step - loss: 0.4864 - acc: 0.8660


[0.48635953664779663, 0.8659999966621399]

In [165]:
%rm -rf /content/Task6
%rm /content/polish.stopwords.txt
%rm /content/training_set_clean_only_tags.txt
%rm /content/training_set_clean_only_text.txt